In [228]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import torch
import random
device = 'cuda' if torch.cuda.is_available() else 'cpu'
import os,sys
opj = os.path.join
from tqdm import tqdm
import acd
from copy import deepcopy
sys.path.append('vae')
from model import init_specific_model
from losses import Loss, _get_log_pz_qz_prodzi_qzCx, _get_log_qz_qzi_perb
from dset import get_dataloaders
from training import Trainer
from utils import *
import pickle as pkl
import pandas as pd
from sim_gaussian_mixture import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [43]:
dirs = ["vary_attr0_seed=10", "vary_attr1_seed=10", "vary_beta0_seed=10"]
results = []
for i in range(len(dirs)):
    # load results
    out_dir = opj("/home/ubuntu/transformation-importance/ex_gaussian_mixture/results", dirs[i])
    fnames = sorted(os.listdir(out_dir))

    results_list = []
    for fname in fnames:
        if fname[-3:] == 'pkl':
            results_list.append(pkl.load(open(opj(out_dir, fname), 'rb')))
    results.append(pd.DataFrame(results_list))
results[2]['beta']

0     0.001
1     0.173
2     0.346
3     0.518
4     0.691
5     0.863
6     1.035
7     1.208
8     1.380
9     1.552
10    1.725
11    1.897
12    2.070
13    2.242
14    2.414
15    2.587
16    2.759
17    2.931
18    3.104
19    3.276
20    3.449
21    3.621
22    3.793
23    3.966
24    4.138
25    4.310
26    4.483
27    4.655
28    4.828
29    5.000
Name: beta, dtype: float64

In [45]:
p.seed = 10
p.beta = np.round(np.linspace(1e-3, 5, 30), 3)[20]
p.num_epochs = 100
p.hidden_dim = 7
p.attr = 0
p.dirname = 'vary_beta0'

# seed
random.seed(p.seed)
np.random.seed(p.seed)
torch.manual_seed(p.seed)

# GET DATALOADERS
(train_loader, train_latents), (test_loader, test_latents) = define_dataloaders(p)

# PREPARES MODEL
model = init_specific_model(orig_dim=p.orig_dim, latent_dim=p.latent_dim, hidden_dim=p.hidden_dim)
model = model.to(device)

# TRAINS
optimizer = torch.optim.Adam(model.parameters(), lr=p.lr)
loss_f = Loss(beta=p.beta, attr=p.attr, alpha=p.alpha, gamma=p.gamma, tc=p.tc, is_mss=True)
trainer = Trainer(model, optimizer, loss_f, device=device)
# trainer(train_loader, test_loader, epochs=p.num_epochs)

In [46]:
# calculate losses
print('calculating losses and metric...')    
rec_loss, kl_loss, mi_loss, tc_loss, dw_kl_loss, attr_loss = calc_losses(model, test_loader, loss_f)
s.reconstruction_loss = rec_loss
s.kl_normal_loss = kl_loss
s.total_correlation = tc_loss
s.mutual_information = mi_loss
s.dimensionwise_kl_loss = dw_kl_loss
if p.attr > 0:
    s.attribution_loss = attr_loss
s.disentanglement_metric = calc_disentangle_metric(model, test_loader).mean()
s.net = model    

calculating losses and metric...


## Local independence

In [246]:
n_data = train_loader.dataset.data.shape[0]

data = iter(train_loader).next().to(device)
recon_data, latent_dist, latent_sample = model(data)

batch_size, latent_dim = latent_sample.shape

log_pz, log_qz, log_qzi, log_prod_qzi, log_q_zCx = _get_log_pz_qz_prodzi_qzCx(latent_sample,
                                                                              latent_dist,
                                                                              n_data,
                                                                              is_mss=True)  

attr_loss = 0
log_q_zCzi = log_qz.view(batch_size, 1) - log_qzi

eps_batch = 50
eps = 0.1
deltas = 2 * eps * torch.rand(eps_batch) - eps

i = 0
perb = torch.zeros(batch_size, latent_dim, eps_batch).to(latent_sample.device)
perb[:,i] = deltas.view(1, eps_batch) * torch.ones(batch_size, 1)
latent_sample_p = latent_sample.unsqueeze(2) + perb

log_qz_p, log_qzi_p = _get_log_qz_qzi_perb(latent_sample_p, latent_dist, n_data, is_mss=True)
log_q_zCzi_p = log_qz_p.view(batch_size, 1, eps_batch) - log_qzi_p    

In [256]:
diff = (log_q_zCzi_p - log_q_zCzi.unsqueeze(2))[:,i,:]

In [263]:
torch.nn.L1Loss()(diff, torch.zeros_like(diff))

tensor(0.0001, device='cuda:0', grad_fn=<L1LossBackward>)

In [265]:
abs(diff).sum()

tensor(0.3766, device='cuda:0', grad_fn=<SumBackward0>)

In [260]:
abs(diff).mean()

tensor(0.0001, device='cuda:0', grad_fn=<MeanBackward0>)